In [1]:
import numpy as np
import pandas as pd
import requests

Goal: Country level rankings over time -- and the factors and sensitivities that determine the rankings. 

In [4]:
requests.get("https://api.climatetrace.org/emissions_by_country?interval=year&sectors=agriculture&sectors=buildings&sectors=extraction&sectors=forests&sectors=manufacturing&sectors=maritime&sectors=oil%20and%20gas&sectors=power&sectors=transport&sectors=waste&since=2010&to=2020").json()

[{'country_full': None,
  'country': 'ABW',
  'quality': None,
  'updated_at': '2020-12-31',
  'emissions': [{'start': '2020-01-01', 'end': '2021-01-01', 'value': 0.0},
   {'start': '2020-01-01', 'end': '2021-01-01', 'value': 1435809.084},
   {'start': '2019-01-01', 'end': '2020-01-01', 'value': 0.0},
   {'start': '2019-01-01', 'end': '2020-01-01', 'value': 1592752.42},
   {'start': '2018-01-01', 'end': '2019-01-01', 'value': 0.0},
   {'start': '2018-01-01', 'end': '2019-01-01', 'value': 1574779.884},
   {'start': '2017-01-01', 'end': '2018-01-01', 'value': 0.0},
   {'start': '2017-01-01', 'end': '2018-01-01', 'value': 1574356.1},
   {'start': '2016-01-01', 'end': '2017-01-01', 'value': 0.0},
   {'start': '2016-01-01', 'end': '2017-01-01', 'value': 1563270.578},
   {'start': '2015-01-01', 'end': '2016-01-01', 'value': 0.0},
   {'start': '2015-01-01', 'end': '2016-01-01', 'value': 1570779.959}]},
 {'country_full': None,
  'country': 'AFG',
  'quality': None,
  'updated_at': '2020-12-31'